# Practica 3

José Delgado, Javier Ortí, Joan Benlloch

1. Estructura de los datos\
*Carpeta images*: Contiene las imágenes originales de los problemas gastrointestinales, que son las entradas (X) del modelo. Estas imágenes tienen características que el modelo necesita aprender para identificar áreas relevantes.\
*Carpeta masks*: Contiene las máscaras de segmentación correspondientes, que son las salidas deseadas (y). Cada máscara tiene el mismo tamaño que su imagen correspondiente y está compuesta por valores binarios (0 y 1):\
0: Representa los píxeles de fondo (no relacionados con el área de interés).\
1: Representa los píxeles que pertenecen a la región de interés (por ejemplo, pólipos o anomalías).\
El modelo se entrena para generar máscaras similares a las de la carpeta masks al procesar imágenes de la carpeta images.

### 1. Preparar el Dataset

transformamos las imágenes y las máscaras en un formato que el modelo pueda procesar:

Carga de imágenes y máscaras: Se leen los archivos .jpg de ambas carpetas, asegurando que cada máscara esté correctamente alineada con su imagen correspondiente.\
Redimensionamiento: Todas las imágenes y máscaras se redimensionan a un tamaño fijo (256x256 en este caso) para que puedan pasar por el modelo. Esto es necesario porque los modelos de deep learning requieren tamaños uniformes.\
Normalización:\
Las imágenes se normalizan dividiendo los valores de los píxeles por 255 (para que estén en el rango [0, 1]).\
Las máscaras permanecen en valores binarios (0 o 1) para representar correctamente las áreas de segmentación.

In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

IMG_HEIGHT, IMG_WIDTH = 256, 256

def load_data(image_dir, mask_dir):
    images, masks = [], []
    for img_name in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_name)
        mask_path = os.path.join(mask_dir, img_name)
        image = img_to_array(load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))) / 255.0
        mask = img_to_array(load_img(mask_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode="grayscale")) / 255.0
        images.append(image)
        masks.append(mask)
    return np.array(images), np.array(masks)

image_dir = 'images'
mask_dir = 'masks'
images, masks = load_data(image_dir, mask_dir)

X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)


In [3]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((800, 256, 256, 3),
 (800, 256, 256, 1),
 (200, 256, 256, 3),
 (200, 256, 256, 1))

### 2. Transfer Learning de ResNet50 para hacer un Autoencoder

3. Entrenamiento del modelo\
El proceso de entrenamiento consiste en mostrar al modelo imágenes de entrada (X) y sus máscaras correspondientes (y), de manera que pueda aprender la relación entre ambas.

Entrada (X): Imagen original (por ejemplo, una imagen de una colonoscopia).\
Salida esperada (y): Máscara correspondiente, donde los píxeles que representan pólipos están marcados con 1.\
En cada iteración:

La imagen pasa por la red convolucional (DeepLab o el modelo definido) que genera una máscara predicha (y_pred) de dimensiones (256x256x1).\
El modelo compara la máscara predicha (y_pred) con la máscara real (y) utilizando la función de pérdida (en este caso, binary cross-entropy).\
Los gradientes calculados por la pérdida ajustan los pesos del modelo para mejorar futuras predicciones.

4. Cómo aprende el modelo\
Durante el entrenamiento:

El modelo extrae características de las imágenes usando capas convolucionales.\
Por ejemplo, identifica patrones relacionados con bordes, texturas y colores que son típicos de las áreas de interés (como los pólipos).\
Las capas de upsampling (deconvoluciones) reconstruyen la máscara a partir de estas características aprendidas, pixel a pixel.\
El modelo ajusta sus pesos para minimizar la diferencia entre la máscara generada (y_pred) y la máscara real (y).

In [8]:
from tensorflow.keras.layers import UpSampling2D, Conv2D

def build_deeplab_model_fixed(input_size=(IMG_HEIGHT, IMG_WIDTH, 3)):
    # Cargar el modelo base (ResNet50)
    base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=input_size)
    base_model.trainable = False

    # Extraer la última capa del modelo base
    x = base_model.output  # Salida típica: (None, 8, 8, 2048)

    # Upsampling para restaurar el tamaño original
    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)  # De 8x8 a 16x16
    x = Conv2D(512, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)  # De 16x16 a 32x32
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D(size=(2, 2), interpolation='bilinear')(x)  # De 32x32 a 64x64
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)

    x = UpSampling2D(size=(4, 4), interpolation='bilinear')(x)  # De 64x64 a 256x256
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    # Salida final con un canal (máscara binaria)
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

    return Model(inputs=base_model.input, outputs=outputs)

model = build_deeplab_model_fixed()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()



Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [9]:
output_shape = model.output_shape
print(f"Output shape of the model: {output_shape}")  # Debería ser (None, 256, 256, 1)


Output shape of the model: (None, 256, 256, 1)


### 3. Entrenar el Modelo

He parado el entrenamiento en la epoch 36

In [10]:
batch_size = 16
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=batch_size
)

Epoch 1/50
50/50 [==============================] - 101s 2s/step - loss: 0.3960 - accuracy: 0.8234 - val_loss: 0.3690 - val_accuracy: 0.8369
Epoch 2/50
50/50 [==============================] - 74s 1s/step - loss: 0.3588 - accuracy: 0.8426 - val_loss: 0.3644 - val_accuracy: 0.8368
Epoch 3/50
50/50 [==============================] - 76s 2s/step - loss: 0.3602 - accuracy: 0.8414 - val_loss: 0.3860 - val_accuracy: 0.8357
Epoch 4/50
50/50 [==============================] - 76s 2s/step - loss: 0.3617 - accuracy: 0.8431 - val_loss: 0.3682 - val_accuracy: 0.8357
Epoch 5/50
50/50 [==============================] - 76s 2s/step - loss: 0.3560 - accuracy: 0.8437 - val_loss: 0.3617 - val_accuracy: 0.8358
Epoch 6/50
50/50 [==============================] - 76s 2s/step - loss: 0.3526 - accuracy: 0.8443 - val_loss: 0.3591 - val_accuracy: 0.8359
Epoch 7/50
50/50 [==============================] - 75s 2s/step - loss: 0.3556 - accuracy: 0.8438 - val_loss: 0.3593 - val_accuracy: 0.8357
Epoch 8/50
50/50 [=

KeyboardInterrupt: 

### 4. Fine-tuning

In [14]:
input_size=(IMG_HEIGHT, IMG_WIDTH, 3)
# Descongelar las últimas capas del modelo base
base_model = tf.keras.applications.ResNet50(weights="imagenet", include_top=False, input_shape=input_size)
base_model.trainable = True
fine_tune_at = 100  # Número de capas a descongelar

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])




In [15]:
#hago una copia del modelo para hacer el fuine tuning
model_fine_tuned = model

In [16]:
history_fine = model_fine_tuned.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=batch_size
)

Epoch 1/20
50/50 [==============================] - 80s 2s/step - loss: 0.3220 - accuracy: 0.8541 - val_loss: 0.3333 - val_accuracy: 0.8437
Epoch 2/20
50/50 [==============================] - 80s 2s/step - loss: 0.3201 - accuracy: 0.8548 - val_loss: 0.3317 - val_accuracy: 0.8443
Epoch 3/20
50/50 [==============================] - 80s 2s/step - loss: 0.3192 - accuracy: 0.8558 - val_loss: 0.3343 - val_accuracy: 0.8430
Epoch 4/20
50/50 [==============================] - 81s 2s/step - loss: 0.3182 - accuracy: 0.8558 - val_loss: 0.3341 - val_accuracy: 0.8437
Epoch 5/20
50/50 [==============================] - 80s 2s/step - loss: 0.3180 - accuracy: 0.8564 - val_loss: 0.3405 - val_accuracy: 0.8427
Epoch 6/20
50/50 [==============================] - 80s 2s/step - loss: 0.3172 - accuracy: 0.8571 - val_loss: 0.3342 - val_accuracy: 0.8439
Epoch 7/20
50/50 [==============================] - 81s 2s/step - loss: 0.3180 - accuracy: 0.8563 - val_loss: 0.3312 - val_accuracy: 0.8449
Epoch 8/20
50/50 [==

### 5. Evaluar y Guardar el Modelo

In [12]:
loss, acc = model.evaluate(X_val, y_val, batch_size=8)
print(f"Validation Loss: {loss}, Validation Accuracy: {acc}")

model.save('deeplabv3_segmentation.h5')


25/25 [==============================] - 1s 45ms/step - loss: 0.3370 - accuracy: 0.8412
Validation Loss: 0.3369520902633667, Validation Accuracy: 0.8412032127380371


In [17]:
# Evaluación del modelo fine-tuned
loss, acc = model_fine_tuned.evaluate(X_val, y_val, batch_size=8)
print(f"Validation Loss: {loss}, Validation Accuracy: {acc}")

model_fine_tuned.save('deeplabv3_segmentation_finetuned.h5')


25/25 [==============================] - 1s 44ms/step - loss: 0.3298 - accuracy: 0.8449
Validation Loss: 0.3297668397426605, Validation Accuracy: 0.8449129462242126
